In [ ]:
# -*- coding: utf-8 -*-
'''
orca-ir Jupyter edition

1. Upload 'orca.out' to the same folder
2. Insert the filename here: imp_data('orca.out')
3. Adjust the plot; use the interactive Matplotlib to select a specific region
4. Save the spectrum by clicking on the 'disk' symbol of the interactive Matplotlib window

'''

# the matplotlip widget in IPython 
%matplotlib widget

import re                                      # regular expressions
import numpy as np                             # summation
import matplotlib.pyplot as plt                # plots
from scipy.signal import find_peaks            # peak detection
from ipywidgets import interactive, HBox, VBox # interactive ui elements

def imp_data(file):
    specstring_start = 'IR SPECTRUM'          #check orca.out from here
    specstring_end = 'The first'              #stop reading orca.out from here
    # 3 empty lists (first one is not really necessary)
    modelist = []
    freqlist = []
    intenslist = []
    try:
        with open(file, "r") as input_file:
            for line in input_file:
                #start exctract text 
                if "Program Version 6" in line:
                    intens_column=3
                if "Program Version 5" in line:
                    intens_column=3
                if "Program Version 4" in line:
                    intens_column=2
                if "Program Version 3" in line:
                    intens_column=2
                if line.startswith(specstring_start):
                    #found IR data in orca.out
                    for line in input_file:
                        #stop exctract text 
                        if line.startswith(specstring_end):
                            break
                        #only recognize lines that start with number
                        #split line into 3 lists mode, frequencies, intensities
                        #line should start with a number
                        if re.search("\d:",line): 
                            modelist.append(int(line.strip().split(":")[0])) 
                            freqlist.append(float(line.strip().split()[1]))
                            intenslist.append(float(line.strip().split()[intens_column]))
    # file not found -> message
    except FileNotFoundError:
        print(f"'{file}'" + " not found")
    if modelist:
        return modelist, freqlist, intenslist
    # IR part not found -> message
    else:
        print(f"'{specstring_start}'" + " not found in" + f"'{file}'")
    
def roundup(x):
    #round to next 100
    return x if x % 100 == 0 else x + 100 - x % 100

def gauss(a,m,x,w):
    # calculation of the Gaussian line shape
    # a = amplitude (max y, intensity)
    # x = position
    # m = maximum/meadian (stick position in x, wave number)
    # w = line width, FWHM
    return a*np.exp(-(np.log(2)*((m-x)/w)**2))

# plot the IR spectrum
def plot_ir(w = 15,                   
            transm_style = True, 
            high_to_low_wn = True, 
            show_stick = True, 
            label_peaks = True,
            show_grid = False,
            show_single_gauss = False):
    # w = line width
    # high_to_low_wn = go from high to low wn 
    # show_stick = show the stick spectrum
    # labe_peaks = label the peaks
    # show_grid = show a grid
    # show_single_gauss = show the single gauss functions

    # always close before draw a new one
    plt.close()

    # list for sum of single gauss functions
    gauss_sum = []
    # position
    freqlist = ir_data[1]
    # intensity
    intenslist = ir_data[2]
    # add some extra space in x 
    wn_add = 120 + w * 2
    # prepare plot
    fig, ax = plt.subplots()
    # plot range in x
    plt_range_x=np.arange(0, max(freqlist) + wn_add, 1)

    # plot single gauss function for every frequency freq if TRUE
    # generate summation of single gauss functions
    for index, freq in enumerate(freqlist):
        if show_single_gauss:
            ax.plot(plt_range_x,gauss(intenslist[index], plt_range_x, freq, w), alpha=0.5) 
            ax.fill_between(plt_range_x,gauss(intenslist[index], plt_range_x, freq, w), alpha=0.5)
        gauss_sum.append(gauss(intenslist[index], plt_range_x, freq, w))
    
    # y values of the gauss summation
    plt_range_gauss_sum_y = np.sum(gauss_sum, axis=0)
    
    # find peaks scipy function, change height for level of detection
    peaks , _ = find_peaks(plt_range_gauss_sum_y, height = 0.1)

    # plot the spectrum
    ax.plot(plt_range_x, plt_range_gauss_sum_y, color="black", linewidth=0.8)

    # show stick spectrum if TRUE
    if show_stick:
        ax.stem(freqlist, intenslist, linefmt="dimgrey", markerfmt=" ", basefmt=" ")

    # title, label, ticks, linear locator
    ax.set_title('IR spectrum', fontweight = 'bold')
    ax.set_xlabel(r'$\tilde{\nu}$ /cm$^{-1}$')  
    ax.get_yaxis().set_ticks([])   
    ax.minorticks_on()
    ax.xaxis.set_major_locator(plt.LinearLocator())

    # transmission or absorption style
    if transm_style:
        plt.ylim(max(plt_range_gauss_sum_y)+max(plt_range_gauss_sum_y)*0.1,0)
        ax.set_ylabel('transmittance')
    else:
        plt.ylim(0,max(plt_range_gauss_sum_y)+max(plt_range_gauss_sum_y)*0.1)
        ax.set_ylabel('intensity')
        label_rel_pos_y=5

    # high to low wn or low to high wn
    if high_to_low_wn:
        plt.xlim(roundup(max(plt_range_x)),0) # round to next 100
    else:
        plt.xlim(0,roundup(max(plt_range_x)))

    # show grid if TRUE
    if show_grid:
        ax.grid(True,which='major', axis='x',color='black',linestyle='dotted', linewidth=0.5)

    # show labels on peaks if TRUE
    if label_peaks:
        for index, txt in enumerate(peaks):
            
            if transm_style:
                #corr_factor - maintain distance from label to peak in transmittance style
                #sensitive to peak label font size
                corr_factor = (4 - len(str(peaks[index])))*3.75
                label_rel_pos_y = -15
                #if one does not care:
                #corr_factor = 0 
            else:
                # not necessary for absorption style
                label_rel_pos_y=5  
                corr_factor = 0
                
            ax.annotate(peaks[index], xy=(peaks[index], plt_range_gauss_sum_y[peaks[index]]),
                        ha = "center", rotation = 90, size = 6,
                        xytext=(0,label_rel_pos_y+corr_factor), textcoords = 'offset points')

    # resize the Matplotlib window
    N = 1.5
    params = plt.gcf()
    plSize = params.get_size_inches()
    params.set_size_inches((plSize[0]*N, plSize[1]*N))

    # tight layout and show
    plt.tight_layout() 
    plt.show()


# upload orca.out in the same folder (location)
# insert the filename; imp_data('orca.out)
ir_data = imp_data('bf43.out')

# make it interactive
interactive_plot = interactive(plot_ir, w = (5, 100, 5), 
                               show_stick = True, 
                               transm_style = True, 
                               high_to_low_wn = True,
                               label_peaks = True,
                               show_grid = False,
                               show_single_gauss = False)

# ui elements (all to last -1)
ui_el = interactive_plot.children[:-1]
# plot window (last)
plot_win = interactive_plot.children[-1]

# ui elements in rows
row1 = HBox(ui_el[:1])
row2 = HBox(ui_el[1:3])
row3 = HBox(ui_el[3:5])
row4 = HBox(ui_el[5:])

# display
interactive_plot = VBox([row1, row2, row3, row4, plot_win])
interactive_plot